<center><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks">unSkript Runbooks</h1>
<div class="alert alert-block alert-success">
<h3 id="Objective"><strong>Objective</strong></h3>
<strong>To get candidate k8s nodes for a given configuration using unSkript actions.</strong></div>
</center><center>
<h2 id="Get-Candidate-k8s-Nodes-For-Given-Configuration">Get Candidate k8s Nodes For Given Configuration</h2>
</center>
<h1 id="Steps-Overview">Steps Overview</h1>
<p>1. Get the matching nodes for a given configuration<code>
</code></p>

<h3 id="Get-candidate-k8s-nodes-for-the-given-configuration">Get candidate k8s nodes for the given configuration</h3>
<p>Here we will use unSkript <strong>Get candidate k8s nodes for the given configuration</strong> action. This action is used to find out matching nodes for a given configuration.</p>
<blockquote>
<p><strong>Input parameters:</strong> <code>attachable_volumes_aws_ebs, cpu_limit, memory_limit, pod_limit</code></p>
</blockquote>
<blockquote>
<p><strong>Output variable:</strong> <code>candidate_nodes</code></p>
</blockquote>
<p>&nbsp;</p>

In [43]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##

import pprint
from typing import Optional

from kubernetes import client
from pydantic import BaseModel, Field
from tabulate import tabulate

pp = pprint.PrettyPrinter(indent=2)


from beartype import beartype
@beartype
def k8s_get_candidate_nodes_for_pods(handle, attachable_volumes_aws_ebs: int = 0, cpu_limit: int = 0, memory_limit: str = "", pod_limit: int = 0):

    coreApiClient = client.CoreV1Api(api_client=handle)

    nodes = coreApiClient.list_node()
    match_nodes = [node for node in nodes.items if
                   (attachable_volumes_aws_ebs < int(node.status.capacity.get("attachable-volumes-aws-ebs", 0))) and
                   (cpu_limit < int(node.status.capacity.get("cpu", 0))) and
                   (pod_limit < int(node.status.capacity.get("pods", 0))) and
                   int(memory_limit.split("Mi")[0]) < (int(node.status.capacity.get("memory").split("Ki")[0]) / 1024)]

    if len(match_nodes) > 0:
        data = []

        for node in match_nodes:
            node_capacity = []
            node_capacity.append(node.metadata.name)
            for capacity in node.status.capacity.values():
                node_capacity.append(capacity)
            data.append(node_capacity)

        print("\n")
        print(tabulate(data, tablefmt="grid", headers=["Name", "attachable-volumes-aws-ebs", "cpu", "ephemeral-storage",
                                                       "hugepages-1Gi", "hugepages-2Mi", "memory", "pods"]))
        return match_nodes`

    pp.pprint("No Matching Nodes Found for this spec")
    return None


task = Task(Workflow())
task.configure(outputName="candidate_nodes")
task.configure(inputParamsJson='''{
    "attachable_volumes_aws_ebs": "int(ebs_limit)",
    "cpu_limit": "int(cpu_limit)",
    "memory_limit": "memory_limit",
    "pod_limit": "int(pod_limit)"
    }''')
task.configure(printOutput=True)

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(k8s_get_candidate_nodes_for_pods, hdl=hdl, args=args)
    if task.output_name != None:
        globals().update({task.output_name: task.output[0]})

if hasattr(task, 'output'):
    if isinstance(task.output, (list, tuple)):
        for item in task.output:
            print(f'item: {item}')
    elif isinstance(task.output, dict):
        for item in task.output.items():
            print(f'item: {item}')
    else:
        print(f'Output for {task.name}')
        print(task.output)
    w.tasks[task.name]= task.output

<h3 id="Conclusion">Conclusion</h3>
<p>In this Runbook, we demonstrated the use of unSkript's k8s legos to run k8s configuration and get the matching nodes for a given configuration (storage, CPU, memory, pod_limit). To view the full platform capabilities of unSkript please visit&nbsp;<a href="https://us.app.unskript.io" target="_blank" rel="noopener">https://us.app.unskript.io</a></p>